In [1]:
# default_exp data.finance

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from libs import is_lib_exists

In [5]:
required_libs = [ ("numpy", "numpy"),
                  ("pandas", "pandas"),
                  ("seaborn", "seaborn"),
                  ("matplotlib", "matplotlib"),
                  ("investpy", "investpy"),
                  ("dateutil", "python-dateutil"),
                ]

In [6]:
for (clz,lib) in required_libs:
    if not is_lib_exists(clz):
        print(f"Installing {lib}")
        !pip install {lib}
    else:
        print(f"{lib} exists")

numpy exists
pandas exists
seaborn exists
matplotlib exists
investpy exists
python-dateutil exists


In [7]:
#export
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import investpy
from pathlib import Path
from dateutil.relativedelta import *
from datetime import *
import calendar
import traceback

In [8]:
%matplotlib inline

In [9]:
# % - line, %% - cell
# %run, %%timeit, %%writefile, %pycat
# %who, %store
# %html 
# %env, %pinfo
# %lsmagic
# %history
# %prun
# %%heat
# https://github.com/lmmx/devnotes/wiki/IPython-'magic'-function-documentation#edit
# %quickref

# https://ipython.readthedocs.io/en/stable/interactive/magics.html#

In [10]:
#%%bash
#for i in {2..10}
#do
#echo $i
#done

In [11]:
from IPython.display import display, Image

# Finance Data

> Functionality to gather finance data.

The classes here provide bases classes used to gather finance data for model training.

In [12]:
#hide
from nbdev.showdoc import *

In [13]:
# Dataset path
DATASET_PATH = Path("../../data")  

# Country to analyze
COUNTRY = "malaysia"

# Dataset
DATASET = DATASET_PATH/f"ftse_klse_investing.csv"

# Stocks summary
DATASET_STOCKS_SUMMARY = DATASET_PATH/f"stocks_summary.csv"

## FTSE KLSE

In [14]:
# Retrieve stock components
dataset = investpy.get_stocks(country=COUNTRY)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   country    929 non-null    object
 1   name       929 non-null    object
 2   full_name  929 non-null    object
 3   isin       929 non-null    object
 4   currency   929 non-null    object
 5   symbol     929 non-null    object
dtypes: object(6)
memory usage: 43.7+ KB


In [15]:
# Save the dataset
dataset.to_csv(DATASET, index=False)

#dataset = pd.read_csv(DATASET, header=0, sep=',', quotechar='"')

In [16]:
display(dataset.head(3))

,country,name,full_name,isin,currency,symbol
0,malaysia,Media Chinese Int,Media Chinese International Ltd,BMG5959D1048,MYR,MDCH
1,malaysia,AMMB,AMMB Holdings Bhd,MYL1015OO006,MYR,AMMB
2,malaysia,CIMB Group,CIMB Group Holdings Bhd,MYL1023OO000,MYR,CIMB


## TODO - stock sector, balance sheet and P&L

In [17]:
def calculate_yield(idx, name, symbol):
    """Calculate stock yield based on latest close price"""
    try:
        df_dividends = investpy.get_stock_dividends(stock=symbol, country=COUNTRY)
        #display(df_dividends)
        df_dividends['Year'] = pd.DatetimeIndex(df_dividends['Date']).year
        # Convert yield percentage to number
        df_dividends['Historical Yield'] =  df_dividends['Yield'].apply(\
                                                     lambda y : \
                                                     float(y.translate(str.maketrans({',': '.', '%': '', '-':'0'}))))
        
        # Aggregate the dividends by years
        df_dividends_summary = df_dividends.groupby(by=['Year'], sort=False, as_index=False).\
                                agg({'Dividend':'sum', 'Historical Yield':'sum', \
                                     'Date': lambda d: ','.join(d.dt.strftime('%Y-%m-%d'))})
        # print(df_dividends.groupby(by=['Year'], sort=False, as_index=False))
        # display(df_dividends_summary[df_dividends_summary['Year'].isin(PAST_3_YEARS)])
        #display(df_dividends_summary)
        
        df_recent_data = investpy.get_stock_recent_data(stock=symbol, country=COUNTRY).reset_index()
        last_close_price = df_recent_data[df_recent_data['Date'].eq(df_recent_data['Date'].max())]['Close'].values[0]
        # display(f"Recent price {last_close_price}")
        # display(df_recent_data)
     
        df_dividends_summary['Name'] = name
        df_dividends_summary['Symbol'] = symbol
        df_dividends_summary['Last Close Price'] = last_close_price
        df_dividends_summary['Calculated Yield'] = df_dividends_summary['Dividend'] / last_close_price * 100
       
        # display(df_dividends_summary)
        
        
        print(f'{idx}. {symbol} - {name}')
        return df_dividends_summary
            
    except Exception as exception:
        print(f"{idx}. Skipping {name} - {symbol}")
        #traceback.print_exc()
        return pd.DataFrame()

In [18]:
def summarize_stocks(df_summary, df):
    """Summarize the stock dividends details"""
    if df is None or df.empty:
        return df_summary
    if df_summary.empty:
        return df
    return df_summary.append(df)

In [19]:
import time

def generate_dataset():
    #!rm -rf {DATASET_STOCK_SUMMARY}
    df_all_dividends = pd.DataFrame()

    #df = calculate_yield(0, "BIMB", "BIMB")
    #df_all_dividends = summarize_stocks(df_all_dividends, df)
    #display(df_all_dividends)

    START_INDEX = 0
    for idx, row in dataset.iloc[START_INDEX:].iterrows():
        name = row['name']
        symbol = row['symbol']
        df = calculate_yield(idx, name, symbol)
        df_all_dividends = summarize_stocks(df_all_dividends, df)
        if (idx % 10 == 0 or idx == (len(dataset) - 1)):
            df_all_dividends.to_csv(DATASET_STOCKS_SUMMARY, mode='a', index=False)
            df_all_dividends = pd.DataFrame()
        time.sleep(3)
    # display(df_all_dividends)

# Generate the dataset again?


In [20]:
dataset = pd.read_csv(DATASET_STOCKS_SUMMARY, header=0, sep=',', quotechar='"')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              2551 non-null   int64  
 1   Dividend          2551 non-null   float64
 2   Historical Yield  2551 non-null   float64
 3   Date              2551 non-null   object 
 4   Name              2551 non-null   object 
 5   Symbol            2551 non-null   object 
 6   Last Close Price  2551 non-null   float64
 7   Calculated Yield  2551 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 159.6+ KB


In [28]:
# Today's date
TODAY = date.today()

# Past 3 years
#PAST_3_YEARS = [ TODAY.year - y for y in range(1,4) ]
PAST_3_YEARS = [ TODAY.year - y for y in range(0,3) ]
print(PAST_3_YEARS)

# Expected YIELD %
EXPECTED_YIELD_PERCENTAGE = 15

[2020, 2019, 2018]


In [29]:
df_dividend_stocks = dataset[
                 (dataset['Year'].isin(PAST_3_YEARS)) & 
                 (dataset['Calculated Yield'] >= EXPECTED_YIELD_PERCENTAGE)]
df_dividend_stocks = df_dividend_stocks.sort_values(by=['Name', 'Year'])

In [30]:
for symbol in df_dividend_stocks['Symbol'].unique():
    display(dataset[dataset['Symbol']==symbol])

,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
571,2020,0.100,5.85,2020-08-12,APM Automotive,APMA,1.71,5.847953
572,2019,0.240,12.14,"2019-09-18,2019-06-11",APM Automotive,APMA,1.71,14.035088
573,2018,0.265,8.15,"2018-09-18,2018-06-06",APM Automotive,APMA,1.71,15.497076
574,2017,0.295,8.49,"2017-09-06,2017-06-06",APM Automotive,APMA,1.71,17.251462
575,2016,0.365,9.99,"2016-09-06,2016-06-02",APM Automotive,APMA,1.71,21.345029
576,2015,0.195,5.00,2015-09-07,APM Automotive,APMA,1.71,11.403509


,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
604,2019,0.0200,11.76,2019-09-26,AYS Ventures,AYSV,0.175,11.428571
605,2018,0.0500,14.74,"2018-08-01,2018-03-14",AYS Ventures,AYSV,0.175,28.571429
606,2017,0.0250,5.95,2017-08-02,AYS Ventures,AYSV,0.175,14.285714
607,2016,0.0283,6.11,"2016-12-06,2016-03-07",AYS Ventures,AYSV,0.175,16.171429


,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
282,2019,0.24,28.24,2019-03-11,Airasia,AIRA,0.9,26.666667
283,2018,0.24,7.74,2018-06-12,Airasia,AIRA,0.9,26.666667
284,2017,0.30,9.13,"2017-09-13,2017-05-23",Airasia,AIRA,0.9,33.333333
285,2016,0.04,1.30,2016-05-26,Airasia,AIRA,0.9,4.444444
286,2015,0.03,1.64,2015-06-01,Airasia,AIRA,0.9,3.333333


,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
2120,2020,0.0448,6.53,"2020-03-24,2020-02-14",Al-Salam Real Estate,ALSA,0.69,6.492754
2121,2019,0.2150,24.52,"2019-12-11,2019-09-23,2019-06-17,2019-02-14",Al-Salam Real Estate,ALSA,0.69,31.159420
2122,2018,0.1215,14.30,"2018-09-18,2018-06-13,2018-02-12",Al-Salam Real Estate,ALSA,0.69,17.608696
2123,2017,0.1225,12.42,"2017-09-13,2017-02-14",Al-Salam Real Estate,ALSA,0.69,17.753623
2124,2016,0.0320,3.05,2016-09-13,Al-Salam Real Estate,ALSA,0.69,4.637681


,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
2525,2020,0.0335,7.98,2020-06-04,AmFIRST,AMFL,0.42,7.976190
2526,2019,0.0786,17.02,"2019-12-03,2019-05-09",AmFIRST,AMFL,0.42,18.714286
2527,2018,0.0815,14.57,"2018-12-03,2018-05-07",AmFIRST,AMFL,0.42,19.404762


...

,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
2507,2020,0.0911,7.41,2020-02-05,UOA REIT,UOAR,1.23,7.406504
2508,2019,0.1846,14.94,"2019-08-06,2019-02-04",UOA REIT,UOAR,1.23,15.008130
2509,2018,0.1684,11.82,"2018-08-09,2018-02-07",UOA REIT,UOAR,1.23,13.691057


,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
2067,2020,0.0680,5.79,"2020-02-17,2020-01-16",VS Industry Bhd,VSID,1.0,6.80
2068,2019,0.1910,16.92,"2019-10-15,2019-07-15,2019-04-10,2019-02-20,20...",VS Industry Bhd,VSID,1.0,19.10
2069,2018,0.2560,13.22,"2018-10-12,2018-07-12,2018-04-10,2018-02-21,20...",VS Industry Bhd,VSID,1.0,25.60
2070,2017,0.1819,8.00,"2017-10-12,2017-07-10,2017-04-26,2017-02-24",VS Industry Bhd,VSID,1.0,18.19
2071,2016,0.2280,17.77,"2016-07-13,2016-04-27,2016-02-24,2016-01-15",VS Industry Bhd,VSID,1.0,22.80
2072,2015,0.0360,0.00,2015-10-12,VS Industry Bhd,VSID,1.0,3.60


,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
2076,2020,0.1110,13.06,"2020-06-12,2020-03-10",Wellcall Holdings Bhd,WCAL,0.83,13.373494
2077,2019,0.2205,19.07,"2019-12-13,2019-09-10,2019-06-12,2019-03-13",Wellcall Holdings Bhd,WCAL,0.83,26.566265
2078,2018,0.2308,18.06,"2018-12-06,2018-09-07,2018-06-08,2018-03-09",Wellcall Holdings Bhd,WCAL,0.83,27.807229
2079,2017,0.3072,16.71,"2017-12-12,2017-09-15,2017-06-08,2017-03-16",Wellcall Holdings Bhd,WCAL,0.83,37.012048
2080,2016,0.3680,17.77,"2016-12-08,2016-09-09,2016-06-08,2016-03-09",Wellcall Holdings Bhd,WCAL,0.83,44.337349
2081,2015,0.1840,3.55,"2015-12-07,2015-09-11",Wellcall Holdings Bhd,WCAL,0.83,22.168675


,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
2455,2020,0.0039,0.37,2020-03-04,YTL Hospitality REIT,YTLR,1.07,0.364486
2456,2019,0.3056,22.95,"2019-12-09,2019-08-14,2019-06-17,2019-03-11",YTL Hospitality REIT,YTLR,1.07,28.560748
2457,2018,0.2912,24.42,"2018-12-05,2018-08-10,2018-06-06,2018-03-08",YTL Hospitality REIT,YTLR,1.07,27.214953
2458,2017,0.2702,22.84,"2017-12-06,2017-08-09,2017-06-07,2017-03-08",YTL Hospitality REIT,YTLR,1.07,25.252336
2459,2016,0.0864,7.20,2016-12-09,YTL Hospitality REIT,YTLR,1.07,8.074766


,Year,Dividend,Historical Yield,Date,Name,Symbol,Last Close Price,Calculated Yield
202,2020,0.170,15.32,"2020-05-27,2020-02-18",Zhulian Corp,ZHCO,1.19,14.285714
203,2019,0.320,21.46,"2019-11-14,2019-08-15,2019-05-14,2019-02-11",Zhulian Corp,ZHCO,1.19,26.890756
204,2018,0.270,19.79,"2018-11-02,2018-08-15,2018-05-14,2018-02-12",Zhulian Corp,ZHCO,1.19,22.689076
205,2017,0.240,13.18,"2017-11-01,2017-08-09,2017-05-08,2017-02-14",Zhulian Corp,ZHCO,1.19,20.168067
206,2016,0.180,12.60,"2016-08-10,2016-05-09,2016-02-15",Zhulian Corp,ZHCO,1.19,15.126050
207,2015,0.065,4.48,2015-10-29,Zhulian Corp,ZHCO,1.19,5.462185


## Testing

In [24]:
investpy.get_index_recent_data(index="KL Construction", country=COUNTRY)

,Open,High,Low,Close,Volume,Currency
Date,,,,,,
2020-11-30,174.52,175.01,172.24,172.24,229508496,MYR
2020-12-01,172.04,176.63,172.03,176.63,313066688,MYR
2020-12-02,176.89,177.30,175.39,177.30,283734784,MYR
2020-12-03,177.76,180.87,177.64,180.87,1041849408,MYR
2020-12-04,181.33,181.55,179.46,181.47,438255808,MYR
2020-12-07,182.45,182.48,181.34,181.88,1009920000,MYR
2020-12-08,183.02,183.32,179.58,179.68,441040096,MYR
2020-12-09,180.92,181.62,180.04,180.51,370906912,MYR
2020-12-10,181.05,181.81,180.49,181.24,232006000,MYR


In [25]:
display(investpy.get_stock_information("BATO", country=COUNTRY))

,Stock Symbol,Prev. Close,Todays Range,Revenue,Open,52 wk Range,EPS,Volume,Market Cap,Dividend (Yield),Average Vol. (3m),P/E Ratio,Beta,1-Year Change,Shares Outstanding,Next Earnings Date
0,BATO,14.16,13.92 - 14.26,2.320000e+09,14.18,9.26 - 15.64,0.93,351300.0,4.000000e+09,0.89 (6.29%),968803.0,15.15,0.49,- 7.45%,285530000.0,17/02/2021


In [26]:
investpy.get_indices(COUNTRY)

,country,name,full_name,symbol,currency,class,market
0,malaysia,MSCI Malaysia Net USD,MSCI Malaysia Net USD,MIMY00000NUS,USD,major_indices,global_indices
1,malaysia,MSCI Malaysia,MSCI Malaysia,MIMY00000PMY,MYR,major_indices,global_indices
2,malaysia,DJ Malaysia USD,Dow Jones Malaysia USD,MYDOWD,USD,major_indices,global_indices
3,malaysia,KL Construction,KL Construction,KLCT,MYR,primary_sectors,world_indices
4,malaysia,DJ Islamic Malaysia Titans 25,Dow Jones Islamic Market Malaysia Titans 25,DJMY25,MYR,other_indices,global_indices
5,malaysia,DJ Islam Malaysia Titans 25 USD,Dow Jones Islamic Market Malaysia Titans 25 USD,DJMY25D,USD,other_indices,global_indices
6,malaysia,FTSE4Good Bursa Malaysia,FTSE4Good Bursa Malaysia,FTF4GBM,MYR,other_indices,global_indices
7,malaysia,FTSE Bursa Malaysia KLCI,FTSE Bursa Malaysia KLCI,FTFBMKLCI,MYR,other_indices,global_indices
8,malaysia,KL Energy,KL Energy,KLEN,MYR,primary_sectors,world_indices
9,malaysia,MSCI Malaysia Gross,MSCI International Malaysia Gross Real time,MIMY00000GUS,USD,other_indices,global_indices
